In [1]:
import os
import ee
import geemap
from pathlib import Path
from datetime import datetime, timedelta , date
import time
import pandas as pd
import csv
import glob
import folium
from IPython.display import display, clear_output
import shutil
import shapefile
import ipywidgets as widgets
from tkinter import Tk, filedialog
import traitlets
from ipyfilechooser import FileChooser

In [2]:
class SelectFilesButton(widgets.Button):
    """A file widget that leverages tkinter.filedialog."""

    def __init__(self):
        super(SelectFilesButton, self).__init__()
        # Add the selected_files trait
        self.add_traits(files=traitlets.traitlets.List())
        # Create the button.
        self.description = "Select Files"
        self.icon = "square-o"
        self.style.button_color = "orange"
        # Set on click behavior.
        self.on_click(self.select_files)

    @staticmethod
    def select_files(b):
        """Generate instance of tkinter.filedialog.

        Parameters
        ----------
        b : obj:
            An instance of ipywidgets.widgets.Button 
        """
        # Create Tk root
        root = Tk()
        # Hide the main window
        root.withdraw()
        # Raise the root to the top of all windows.
        root.call('wm', 'attributes', '.', '-topmost', True)
        # List of selected fileswill be set to b.value
        b.files = filedialog.askopenfilename(multiple=True,filetypes=[('shp','.shp')])

        b.description = "Files Selected"
        b.icon = "check-square-o"
        b.style.button_color = "lightgreen"

In [3]:
# After executing this line of code for the first use, you can get the authentication number linked to Google.
Map = geemap.Map()
# Authenticate the Google earth engine with google account
ee.Initialize() 

In [4]:
# give the shp file
my_button = SelectFilesButton()
my_button # This will display the button in the context of Jupyter Notebook

SelectFilesButton(description='Select Files', icon='square-o', style=ButtonStyle(button_color='orange'))

In [5]:
shp = shapefile.Reader("".join(my_button.files))
b = []
for i in range(len(shp.fields)):
    a = shp.fields[i][0]
    b.append(a)

# give the shapefile name

file_name = widgets.Dropdown(
options=b,
description='Regional category')

file_name

Dropdown(description='Regional category', options=('DeletionFlag', 'STATE'), value='DeletionFlag')

In [6]:
# give the star date and end date

star = widgets.DatePicker(
    description='Pick a Star Date',
    disabled=False
)
end = widgets.DatePicker(
    description='Pick a End Date',
    disabled=False
)

widgets.HBox([star, end])

In [7]:
# give the bands
#
band_name = widgets.SelectMultiple(
options=['LST_Day','LST_Night','LST_Mean'],
description='Band',
)

band_name

SelectMultiple(description='Band', options=('LST_Day', 'LST_Night', 'LST_Mean'), value=())

In [8]:
statics =widgets.Dropdown(
    options=['MEAN','MAXIMUM', 'MINIMUM', 'MEDIAN', 'STD', 'VARIANCE', 'SUM'],
    value='MEAN',
    description='Statistics')

statics

Dropdown(description='Statistics', options=('MEAN', 'MAXIMUM', 'MINIMUM', 'MEDIAN', 'STD', 'VARIANCE', 'SUM'),…

In [9]:
# give the output floder and flie name
folder = FileChooser()
display(folder)


FileChooser(path='C:\Users\Yang Hsiu\Downloads\satellite python code-20220518T062225Z-001\satellite python cod…

In [10]:
# create folder
folder_name = 'data_all'

# create folder name : data_all
if os.path.isdir(folder_name) == True:
    shutil.rmtree(folder_name, ignore_errors=True)
    os.makedirs(folder_name)
else:
    os.makedirs(folder_name)

In [11]:
states = geemap.shp_to_ee("".join(my_button.files))

def lst_filter(image):
    
    qaday = image.select(['QC_Day']); 
    qanight = image.select(['QC_Night']); 
    dayshift = qaday.rightShift(6);
    nightshift = qanight.rightShift(6);
    daymask = dayshift.lte(2);
    nightmask = nightshift.lte(2);
    #dayshift1 = qaday.leftShift(6);
    #dayshift2 = qaday.rightShift(6);
    #daymask = dayshift2.eq(0);
    #nightshift1 = qanight.leftShift(6);
    #nightshift2 = qanight.rightShift(6);
    #nightmask = nightshift2.eq(0);  
    outimage = ee.Image(image.select(['LST_Day_1km', 'LST_Night_1km']));
    outmask = ee.Image([daymask, nightmask]);
    return outimage.updateMask(outmask); 





def last_day_of_month(any_day):
    next_month = any_day.replace(day=28) + timedelta(days=4)  # this will never fail
    return next_month - timedelta(days=next_month.day)

def monthlist(begin,end):
    #begin = datetime.datetime.strptime(begin, "%Y-%m-%d")
    #end = datetime.datetime.strptime(end, "%Y-%m-%d")

    result = []
    while True:
        if begin.month == 12:
            next_month = begin.replace(year=begin.year+1,month=1, day=1)
        else:
            next_month = begin.replace(month=begin.month+1, day=1)
        if next_month > end:
            break
        result.append ([begin.strftime("%Y-%m-%d"),last_day_of_month(begin).strftime("%Y-%m-%d")])
        begin = next_month
    result.append ([begin.strftime("%Y-%m-%d"),end.strftime("%Y-%m-%d")])
    return result


def last_day_of_year(any_day):
    next_year = any_day.replace(year=any_day.year, month=12, day = 31)  # this will never fail
    return next_year

def yearlist(begin,end):
    #begin = datetime.datetime.strptime(begin, "%Y-%m-%d")
    #end = datetime.datetime.strptime(end, "%Y-%m-%d")

    result = []
    while True:
        if begin.year < end.year:
            next_year = begin.replace(year=begin.year+1,month=1, day=1)
        else:
            next_year = end
        if next_year == end:
            break
        result.append ([begin.strftime("%Y-%m-%d"),last_day_of_year(begin).strftime("%Y-%m-%d")])
        begin = next_year
    result.append ([begin.strftime("%Y-%m-%d"),end.strftime("%Y-%m-%d")])
    return result

def trans_date(input_date):
    t1 = input_date
    t2 = datetime.strptime(t1, '%Y_%m_%d').strftime('%Y/%m/%d')
    return t2



In [12]:
def lst_day(image):
    
    lst_day = image.select('LST_Day_1km').multiply(0.02).subtract(273.15).rename("LST_Day")
    image = image.addBands(lst_day)

    return(image)

def lst_night(image):
    
    lst_night = image.select('LST_Night_1km').multiply(0.02).subtract(273.15).rename("LST_Night")
    image = image.addBands(lst_night)

    return(image)

def lst_mean(image):
    lst_mean = image.expression(
    '(day + night) / 2', {
    'day': image.select('LST_Day'),
    'night': image.select('LST_Night')}).rename('LST_Mean');

    return image.addBands(lst_mean);

#Combine date csv files and combine them according to different statistical values.

def cbind(statics):

    all_files = glob.glob(os.path.join(folder_name,"Modis_{}*.csv".format(statics)))

    df_from_each_file = (pd.read_csv(f, sep = ",") for f in all_files)
    df_merged = pd.concat(df_from_each_file, ignore_index = True)
    if 'LST_Day' in df_merged.columns.tolist():
        df_merged.rename(columns={'LST_Day' : 'LST_Day_' + str(statics)}, inplace = True)
    else:
        pass
    if 'LST_Night' in df_merged.columns.tolist():
        df_merged.rename(columns={'LST_Night' : 'LST_Night_' + str(statics)}, inplace = True)
    else:
        pass
    if 'LST_Mean' in df_merged.columns.tolist():
        df_merged.rename(columns={'LST_Mean' : 'LST_Mean_' + str(statics)}, inplace = True)
    else:
        pass

    df_merged.to_csv(folder.selected + '.csv')
    
    shutil.rmtree(folder_name, ignore_errors=True)

""" 
Note that here, the merged file path cannot be the same as the single file path, and must be placed in different folders, 
so this time the following line of code is to output the files to another folder.
"""


' \nNote that here, the merged file path cannot be the same as the single file path, and must be placed in different folders, \nso this time the following line of code is to output the files to another folder.\n'

In [13]:
def zonal(statics):
    
    time_list =monthlist(star.value, end.value)
    
    for i in range(0,len(time_list)):
        star_time = time.time()
        
        if os.path.isfile(folder_name +'Modis_{}_{}.csv'.format(statics,time_list[i])) == True:
                
                continue
            
        else:
            clear_output(wait=True)
            Modis = ee.ImageCollection('MODIS/006/MOD11A2') \
                      .filter(ee.Filter.date(time_list[i][0],datetime.strptime(time_list[i][1],"%Y-%m-%d")+ timedelta(days=1))) \
                      .filterBounds(states) \
                      .map(lst_filter) \
                      .map(lst_day) \
                      .map(lst_night) \
                      .map(lst_mean) \
                      .select(list(band_name.value)) \
                      .map(lambda image: image.clip(states))


            Modis = Modis.toBands()
            out_dir = os.path.expanduser(folder_name)
            out_dem_stats = os.path.join(out_dir, 'Modis_{}_{}.csv'.format(statics,time_list[i]))

            if not os.path.exists(out_dir):
                os.makedirs(out_dir)

            geemap.zonal_statistics(Modis, states, out_dem_stats, statistics_type=statics, scale=1000)
            data_temp = pd.read_csv(out_dem_stats)
            column_name_list = data_temp.columns.tolist()
            c = []
            d = []
            for k in zip(column_name_list[:]):
                c.append(k[0][0])
                d.append(k[0])
            data = []
            for j in range(0, len(column_name_list),len(band_name.value)):            
                if all(m.isdigit() for m in c[j:j+len(band_name.value)]) == True:
                        
                    df = data_temp.loc[:,d[j:j+len(band_name.value)]]
                    df[file_name.value] = data_temp.loc[:,[file_name.value]]
                    text = column_name_list[j][0:10]
                    df.insert(0, 'Date', '')
                    df['Date'] = trans_date(text)
                    df.insert(1, 'Doy', '')
                    df['Doy'] = datetime.strptime(text, '%Y_%m_%d').strftime('%j')
                    colnames=['Date','Doy']
                    colnames.extend(list(band_name.value))
                    colnames.append(file_name.value)
                    df.columns=[colnames]
                    data.append(df)
                else:
                    continue
                        
            appended_data = pd.concat(data, axis=0,ignore_index = True)
            #cols = appended_data.columns.to_list()
            #cols.insert(len(appended_data.columns), cols.pop(cols.index(file_name)))
            #appended_data = appended_data[cols]
                
            appended_data.to_csv(out_dem_stats,index=False)#Output the file with date and doy back
            
            end_time = time.time()
            print(end_time - star_time)
            time.sleep(1)

In [14]:
zonal(statics.value)
cbind(statics.value)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Yang Hsiu\Downloads\satellite python code-20220518T062225Z-001\satellite python code\zonal statistic\data_all\Modis_MEAN_['2019-01-01', '2019-01-31'].csv
29.8183856010437
